In [20]:
import pandas as pd

In [21]:
data = pd.read_csv(
    'data.csv'
)

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232725 entries, 0 to 232724
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   genre             232725 non-null  object 
 1   artist_name       232725 non-null  object 
 2   track_name        232724 non-null  object 
 3   track_id          232725 non-null  object 
 4   popularity        232725 non-null  int64  
 5   acousticness      232725 non-null  float64
 6   danceability      232725 non-null  float64
 7   duration_ms       232725 non-null  int64  
 8   energy            232725 non-null  float64
 9   instrumentalness  232725 non-null  float64
 10  key               232725 non-null  object 
 11  liveness          232725 non-null  float64
 12  loudness          232725 non-null  float64
 13  mode              232725 non-null  object 
 14  speechiness       232725 non-null  float64
 15  tempo             232725 non-null  float64
 16  time_signature    23

In [23]:
data.loc[data['popularity'] > 90, ['track_name', 'artist_name', 'popularity', 'genre']]

,track_name,artist_name,popularity,genre
9026,"break up with your girlfriend, i'm bored",Ariana Grande,99,Dance
9027,7 rings,Ariana Grande,100,Dance
9028,Without Me,Halsey,97,Dance
9029,needy,Ariana Grande,92,Dance
9030,NASA,Ariana Grande,91,Dance
...,...,...,...,...
138922,Amanece,Anuel Aa,92,Reggaeton
138931,Adictiva,Daddy Yankee,91,Reggaeton
152266,Close To Me (with Diplo) (feat. Swae Lee),Ellie Goulding,92,R&B
152287,Don't Call Me Up,Mabel,91,R&B


# EDA:

In [24]:
data['genre'] = data['genre'].replace({
        "Children’s Music": "Children's Music",
        "Movie": "Soundtrack",
        "Opera": "Classical"
    })
data['genre'].unique()

array(['Soundtrack', 'R&B', 'A Capella', 'Alternative', 'Country',
       'Dance', 'Electronic', 'Anime', 'Folk', 'Blues', 'Classical',
       'Hip-Hop', "Children's Music", 'Rap', 'Indie', 'Pop', 'Reggae',
       'Reggaeton', 'Jazz', 'Rock', 'Ska', 'Comedy', 'Soul', 'World'],
      dtype=object)

In [25]:
from src.funcseda import music_eda
eda = music_eda(data)
eda.do_encoding()
eda.data = eda.data.sample(frac = 0.18, ignore_index=True)
eda.data = eda.encode_track_id()  

In [26]:
data = eda.data

In [27]:
counts_dict = data['genre'].value_counts(dropna=False).to_dict()
sorted_by_key = dict(sorted(counts_dict.items(), reverse=False))
sorted_by_key

{1: 285,
 3: 124,
 4: 117,
 6: 30,
 8: 163,
 9: 34,
 11: 541,
 13: 692,
 14: 291,
 15: 10,
 16: 1270,
 17: 25,
 19: 18,
 20: 583,
 21: 11,
 23: 100}

In [28]:
counts_dict = data['popularity'].value_counts(dropna=False).to_dict()
sorted_by_key = dict(sorted(counts_dict.items(), reverse=False))
sorted_by_key

{62: 545,
 63: 466,
 64: 438,
 65: 357,
 66: 350,
 67: 306,
 68: 246,
 69: 229,
 70: 215,
 71: 192,
 72: 161,
 73: 130,
 74: 119,
 75: 86,
 76: 87,
 77: 97,
 78: 48,
 79: 45,
 80: 32,
 81: 33,
 82: 20,
 83: 14,
 84: 21,
 85: 13,
 86: 12,
 87: 6,
 88: 3,
 89: 4,
 91: 6,
 92: 2,
 93: 4,
 94: 1,
 95: 1,
 96: 1,
 97: 2,
 99: 1,
 100: 1}

In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4294 entries, 0 to 4293
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   genre             4294 non-null   int64  
 1   artist_name       4294 non-null   object 
 2   track_name        4294 non-null   object 
 3   popularity        4294 non-null   int64  
 4   acousticness      4294 non-null   float64
 5   danceability      4294 non-null   float64
 6   duration_ms       4294 non-null   int64  
 7   energy            4294 non-null   float64
 8   instrumentalness  4294 non-null   float64
 9   key               4294 non-null   int64  
 10  liveness          4294 non-null   float64
 11  loudness          4294 non-null   float64
 12  mode              4294 non-null   int64  
 13  speechiness       4294 non-null   float64
 14  tempo             4294 non-null   Int64  
 15  time_signature    4294 non-null   int64  
 16  valence           4294 non-null   float64


In [30]:
from src.this_mapping import supported_genres

# Создаём маппинг индекс → название жанра
idx_to_genre = {i: name for i, name in enumerate(supported_genres)}

# Декодируем числовой жанр в название
data['genre_name'] = data['genre'].map(idx_to_genre)

# Проверяем, нет ли NaN (некорректных жанров)
if data['genre_name'].isna().any():
    raise ValueError("Обнаружены некорректные значения жанра!")

# Группируем по жанру и считаем СУММАРНУЮ популярность (можно заменить на .mean() для средней)
genre_pop_sum = (
    data.groupby('genre_name')['popularity']
    .sum()  # или .mean(), если хочешь среднюю популярность трека в жанре
    .sort_values(ascending=False)
)

# Выводим как словарь (жанр → суммарная популярность)
genre_pop_dict = genre_pop_sum.to_dict()
for genre, total_pop in genre_pop_dict.items():
    print(f"{genre}")

Pop
Rap
Rock
Hip-Hop
R&B
Indie
Folk
Alternative
Country
Soul
Blues
Electronic
Reggae
Jazz
Ska
Classical


## Каждый значимый признак закодирован. ЕDA окончено

In [24]:
data.to_csv('edadata.csv', index=False)